In [6]:
'''
Author: Vighnesh Ghantasala 
Github: https://github.com/VighneshGhantasala/
'''

import pyautogui as pg
from PIL import ImageGrab, ImageOps
import time
import numpy as np
import cv2

board_val = 0
retry = 0
LIFE = 0
R_VAL = 1455 #normalized value sum of retry button pixels
reward = 1
states = [0]

#Q-matrix
old_q = np.zeros((20,3)) 
q = np.zeros((20,3))
for i in range(q.shape[0]):
    q[i,] = [0 ,1 ,1.5]
    np.random.shuffle(q[i,])
    old_q[i,] = q[i,]


class Cordinates():
    #You may need to  tweak these according your screen resolution
    replay = (329,270,360,301) #replay button coordinates
    replayBtn = (340,300) #approx center coordinates for replay button
    dino = (105,313) #Dino coordinates Standing(86,290) sitting(105,313)
    box = (93,290,159,330) #A rect in-front of our Agent 
def restart():
    pg.click(Cordinates.replayBtn)

def give_reward():
    global reward
    global R_VAL
    c = Cordinates()
    replay = ImageGrab.grab(c.replay) #get replay button position
    replay = np.array(replay)
    if int(replay.sum()/255) == R_VAL: #if replay button is seen then a life has beeen consumed
        reward = 0 #reward 0 when dead
    else:
        reward = 1
def do_none():
    pg.keyDown("left")
    time.sleep(0.1)
    print("Do nothing")
    pg.keyUp("left")
    give_reward()
def jump():
    pg.keyDown("space")
    time.sleep(0.1)
    print("Jump")
    pg.keyUp("space")
    give_reward()


def duck(): #crouch
    
    pg.keyDown("down")
    time.sleep(0.1)
    print("duck")
    pg.keyUp("down")
    give_reward()

def imgGrab():
    global board_val
    global retry
    global reward
    global R_VAL
    c = Cordinates()
    replay = ImageGrab.grab(c.replay) #get replay button position
    replay = np.array(replay)
    if int(replay.sum()/255) == R_VAL: #if replay button is seen then a life has beeen consumed
        print("Life consumed")
        retry += 1

    #print(reward)
    img = ImageGrab.grab(c.box)
    gray = ImageOps.grayscale(img)
    a = np.array(gray.getcolors())
    return a.sum()



In [7]:
gamma = 0.8 #Discount factor
score = 0 # Initial score value
states = np.array(states)
LIFE = 20 #no.of lifes for our Agent(Dino)

def choose_action(state):
    q_row_index = np.where(state == states)[0][0] #Get the row with the state
    action = np.argmax(q[q_row_index,])
    return action

def update_q(current_state,action,reward):
    '''
    Q-Learning algorithm: 
    Q[current_state,action] = rewards[current_state,action] + gamma * max[Q[current_state,action]]    
    
    '''
    
    global gamma
    global alpha
    state_index = np.where(current_state == states)[0][0]
    max_val = np.max(q[state_index,]) #This is the action we took
    q[state_index,action] = reward + gamma * max_val # Algorithm

def run():
    global states
    global score
    global q
    global reward
    while True:
        s = imgGrab()
        #add new states to the states table
        if(len(np.where(states == s)[0]) < 1):
            states = np.append(states,s)

        #print(reward)

#         print(s)
        if retry == 1:
            #print("Game Over")
            break
        if s != 2895 and s!=2640:
            try:
                act = choose_action(s)
                print("action for {} is {}".format(s,act))
            except Exception as e:
                pass
            score += 1
            #print(s)

            #Perform action performed based on Q-table
            #actions = {0:do nothing ,1:jump ,2:duck}
            if act == 0:
                do_none()
                print(reward)
                update_q(s,act,reward)
            elif act == 1:
                jump()
                print(reward)
                update_q(s,act,reward)
            elif act == 2:
                duck()
#                 print(reward)
                print(reward)
                update_q(s,act,reward)
            time.sleep(0.1)
            
        if score == 50:
            print("Score Reached")
            break

for i in range(LIFE):
    if score == 50:
        break
    else:
        print("Score of run:",score)
        score = 0
        print("Life:",i)
        retry = 0
        restart()
        run()

Score of run: 0
Life: 0
action for 2978 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 3227 is 1
Jump
1
action for 2978 is 1
Jump
1
action for 2978 is 1
Jump
0
Life consumed


In [14]:
#Random Testing area 


def stats(): #prints stats like Q-matrix and states
    print("len & num states:",len(states),states)

    print("Q-matrix:\n",q[:len(states)])

stats()
print("\n\n initial Q-matrix\n",old_q[:len(states)] )

# In-case u want to save ur Q-matrix
np.save("Dino_Q_matrix.npy",q)

len & num states: 5 [   0 2895 2978 3227 3144]
Q-matrix:
 [[0.         1.5        1.        ]
 [1.         0.         1.5       ]
 [1.         3.99876205 0.        ]
 [1.         4.96472933 0.        ]
 [0.         2.44       0.96      ]]


 initial Q-matrix
 [[0.  1.5 1. ]
 [1.  0.  1.5]
 [1.  1.5 0. ]
 [1.  1.5 0. ]
 [0.  1.  1.5]]


In [12]:
#Run this cell to obtain R_VAL (Just a helper cell)
import pyautogui as pg
from PIL import ImageGrab, ImageOps
import time
import numpy as np
import cv2


c = Cordinates()
#box = (c.dino[0]+70,290,c.dino[0]+120,c.dino[1]+5)
img = ImageGrab.grab(c.replay)
img = np.array(img)
board_val = int(img.sum()/255)
print(board_val)
cv2.imshow("img",np.array(img))

cv2.waitKey(0)
cv2.destroyAllWindows()

'''
A sample Q-matrix

len & states: 5 [   0 2895 2978 3227 3144]
Q-matrix:
 [[0.         1.5        1.        ]
 [1.         0.         1.5       ]
 [1.         3.99876205 0.        ]
 [1.         4.96472933 0.        ]
 [0.         2.44       0.96      ]]

initial Q-matrix: 
 [[0.  1.5 1. ]
 [1.  0.  1.5]
 [1.  1.5 0. ]
 [1.  1.5 0. ]
 [0.  1.  1.5]]

'''
print(" ")

1455
 
